In [1]:
import os
import shutil
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def prepare_dataset(images_dir, labels_dir, output_dir, train_ratio=0.8,
                    train_txt_path="/content/drive/MyDrive/INVESTIGACIÓN UAM/ESTACIÓN DE MONITOREO/Entrenamiento/train.txt",
                    val_txt_path="/content/drive/MyDrive/INVESTIGACIÓN UAM/ESTACIÓN DE MONITOREO/Entrenamiento/val.txt",
                    move_files_instead_of_copy=False):
    """
    Particiona un dataset de imágenes+etiquetas en train/ y val/, y genera train.txt y val.txt.

    Parámetros:
    -----------
    images_dir : str
        Carpeta donde están TODAS las imágenes originales.
    labels_dir : str
        Carpeta donde están TODAS las etiquetas (mismo nombre de la imagen, terminación .txt).
    output_dir : str
        Directorio donde se creará 'train/images', 'train/labels', 'val/images' y 'val/labels'.
    train_ratio : float
        Porcentaje de datos que irán a entrenamiento. Ej: 0.8 -> 80% train, 20% val.
    train_txt_path : str
        Ruta donde se creará/guardará el archivo train.txt.
    val_txt_path : str
        Ruta donde se creará/guardará el archivo val.txt.
    move_files_instead_of_copy : bool
        Si True, moverá (shutil.move) los archivos. Si False, los copiará (shutil.copy2).

    Regresa:
    --------
    None (genera la estructura de carpetas y los .txt)
    """
     # 1. Crear subcarpetas
    train_images_dir = os.path.join(output_dir, "train/images")
    train_labels_dir = os.path.join(output_dir, "train/labels")
    val_images_dir   = os.path.join(output_dir, "val/images")
    val_labels_dir   = os.path.join(output_dir, "val/labels")

    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(val_images_dir, exist_ok=True)
    os.makedirs(val_labels_dir, exist_ok=True)

    # 2. Listar imágenes
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(image_extensions)]
    random.shuffle(image_files)

    # 3. Partir entre train y val
    train_count = int(len(image_files) * train_ratio)
    train_files = image_files[:train_count]
    val_files   = image_files[train_count:]

    # 4. Funciones de copia/movimiento
    def copy_or_move(src, dst):
        if move_files_instead_of_copy:
            shutil.move(src, dst)
        else:
            shutil.copy2(src, dst)

    # 5. Mover/copiar imágenes+labels para TRAIN
    for img_name in train_files:
        src_img = os.path.join(images_dir, img_name)
        base_name, _ = os.path.splitext(img_name)
        label_name = base_name + ".txt"
        src_label = os.path.join(labels_dir, label_name)

        dst_img   = os.path.join(train_images_dir, img_name)
        dst_label = os.path.join(train_labels_dir, label_name)

        copy_or_move(src_img,   dst_img)
        copy_or_move(src_label, dst_label)

    # 6. Mover/copiar imágenes+labels para VAL
    for img_name in val_files:
        src_img = os.path.join(images_dir, img_name)
        base_name, _ = os.path.splitext(img_name)
        label_name = base_name + ".txt"
        src_label = os.path.join(labels_dir, label_name)

        dst_img   = os.path.join(val_images_dir, img_name)
        dst_label = os.path.join(val_labels_dir, label_name)

        copy_or_move(src_img,   dst_img)
        copy_or_move(src_label, dst_label)

    # 7. Crear train.txt y val.txt con rutas absolutas
    #    (Darknet requiere la ruta completa de cada imagen)
    with open(train_txt_path, 'w') as f:
        for img_name in train_files:
            f.write(os.path.join(os.path.abspath(train_images_dir), img_name) + '\n')

    with open(val_txt_path, 'w') as f:
        for img_name in val_files:
            f.write(os.path.join(os.path.abspath(val_images_dir), img_name) + '\n')

    # 8. Mensajes finales
    print(f"Total de imágenes encontradas: {len(image_files)}")
    print(f"-> Entrenamiento (train): {len(train_files)} imágenes")
    print(f"-> Validación (val): {len(val_files)} imágenes")
    print(f"\nArchivos generados:\n  {train_txt_path}\n  {val_txt_path}")
    print("¡Partición completada con éxito!")

In [4]:
images_dir = "/content/drive/MyDrive/INVESTIGACIÓN UAM/ESTACIÓN DE MONITOREO/Images"
labels_dir = "/content/drive/MyDrive/INVESTIGACIÓN UAM/ESTACIÓN DE MONITOREO/Images"
output_dir = "/content/drive/MyDrive/INVESTIGACIÓN UAM/ESTACIÓN DE MONITOREO/dataset"
train_txt = "/content/drive/MyDrive/INVESTIGACIÓN UAM/ESTACIÓN DE MONITOREO/Entrenamiento/train.txt"
val_txt   = "/content/drive/MyDrive/INVESTIGACIÓN UAM/ESTACIÓN DE MONITOREO/Entrenamiento/val.txt"

In [5]:
prepare_dataset(
    images_dir=images_dir,
    labels_dir=labels_dir,
    output_dir=output_dir,
    train_ratio=0.8,  # 80% entrenamiento, 20% validación
    train_txt_path=train_txt,
    val_txt_path=val_txt,
    move_files_instead_of_copy=False  # Si quieres mover en lugar de copiar, pon True
)

Total de imágenes encontradas: 1506
-> Entrenamiento (train): 1204 imágenes
-> Validación (val): 302 imágenes

Archivos generados:
  /content/drive/MyDrive/INVESTIGACIÓN UAM/ESTACIÓN DE MONITOREO/Entrenamiento/train.txt
  /content/drive/MyDrive/INVESTIGACIÓN UAM/ESTACIÓN DE MONITOREO/Entrenamiento/val.txt
¡Partición completada con éxito!
